# Data Wrangling
### `! git clone https://github.com/ds4e/wrangling`

## Data Wrangling
- Today, we just want to open the data and learn the basic steps of interacting with it
- Before we can do even the most basic analysis, there's typically a lot of work to get the data in a state where it can be used
- This kind of pre-analysis cleaning and management is called **Data Wrangling** or **Data Cleaning**

## Data Wrangling
- According to a survey by CloudFlower and popularized by Forbes, about 82% of data scientists' time is spent on cleaning and organizing data:

![Time use](./src/timespent.jpg)

"76% of data scientists view data preparation as the least enjoyable part of their work."

## Files Containing Data
- These are the most common file formats I am aware of, and what programs or languages created them:

| File Format Extension | Likely Source |
| :---: | :---:|
|.csv | Any |
|.Rda, .Rdata | R |
|.dat | SAS |
|.sav | SPSS |
|.dta | Stata |
|.mat | MATLAB |
|.xls, .xlsx | Excel |
|.json | JavaScript Object Notation|
|.parquet | Apache parquet format|
|.orc | Optimized Row Columnar, for big data |

- Unless you have a good reason, .csv is probably the most accessible format for most people
- More modern formats like parquet and orc save space by storing values in a clever way. If a column is just 0's and 1's with mostly 0's for example, they'll just track the indices where a 1 appears, rather than storing the whole column.

## Codebooks
- Data come with documentation, which I am going to generically call a *codebook* or a *data dictionary*
- It might be a formal codebook, or it might just be the survey itself that respondents filled out, or it might just be the html/javascript code that created the web page that captured the responses
- It's easy to make mistakes without a codebook:
    - `Employees` might be a `True/False` about whether the firm has any employees at all, or it might be the number of employees it employs, and so on

## Opening Data
- In order to work with our data, we need software that handles dataframes. Python does not automatically do this.

`import pandas as pd`

- Open a notebook, and write 


| Method | Usage |
| :---: | :---:|
| `df = pd.read_csv(path)` | Open a generic .csv |
| `df = pd.read_csv(<filename>, encoding = 'latin1')` | Warning about UTF-8 encoding |
| `df = pd.read_csv(<filename>, sep='\t')` | Tab-separated file, (or pipe `\|`, etc.) |
| `df = pd.read_parquet(<filename>,engine='fastparquet)` | Open parquet file; `pip install fastparquet` if necessary |

- Load the `airbnb_NYC.csv` data

## Data 
- A data point or datum is the intersection of three things:
    1. **Observation**: The person, place, or thing being considered
    2. **Variable**: A **numeric** (price, age) or **categorical** (color, brand) representation of some aspect of the observation
    3. **Value**: The **number** (for numeric) or **label** (for categorical) recorded
- A collection of observations is typically represented in a **dataframe**: An $N$-observations by $L$-variables rectangular matrix of information
- So we can have `car = (price, age, color, brand)`, or `patient = (blood_pressure, A1C, creatine_phosphokinase)`, etc.

## Summarizing the Data Frame

- What are the basic features of the data frame?

| Method | Usage |
| :---: | :---:|
| `df.shape` | Number of rows and columns |
| `df.columns` | Names of the variables |
| `df.dtypes` | Types of the variables |
| `df.head(n)` | First $n$ rows |
| `df.tail(n)` | Last $n$ rows |

## Summarizing a Variable

- Great, our data are loaded. Let's dig into these variables.
- What are the basic features of a variable, `var` ?

| Method | Usage |
| :---: | :---:|
| `df[var].unique()` | List of values it takes |
| `df[var].value_counts()` | Counts of each value |
| `df[var].plot.hist()`, `df[var].plot.kde()` | Histogram of values |
| `df[var].describe()` | Statistical summary |

- I personally like `skimpy` as the solution for getting a quick overview of all variables `skim(df)`

## Missing Values
- For a given observation and variable, if a value is not recorded, that value is **missing**
- Pandas records these as `np.nan` or `None` internally 

| Method | Usage |
| :---: | :---:|
| `df['var_na'] = df[var].isna()` | Missing value indicator |

- The fact that the value is missing is **information**. If data are systematically missing, we might be (very) worried. Some examples:
    - Skip Patterns: Questions are only asked in a survey if previous questions are affirmative
    - Selection Bias: Wages are not observed for people out of the labor force; people who make very high or very low wages might not be as inclined to search for work
    - Bond: Bonds in VA are only recorded if they are strictly positive; no one is recorded as having a bond of 0. 

![Survivor Bias](./src/plane.png)

## Cleaning a Numeric Variable

- Many numeric variables are erroneously read in by Pandas as categorical
- This typically happens because either the comma is retained in numbers like $1,000$, or $'s appear in the variable, or the missing values are recorded as text like `NA`

| Method | Usage |
| :---: | :---:|
| 1. `df[var].str.replace(pattern,replace)` | Substitute `pattern` with `replace`, like `,` or `$` with ` `  |
| 2. `pd.to_numeric(df[var],errors='coerce')` | Cast `var` to numeric, when possible, or replace with `np.nan`  |

## Imputation

- So you have those `np.nan` values left in the data frame, what do you do with them?
- It depends on the analytics you plan to do
- But here's a reasonable for now

| Method | Usage |
| :---: | :---:|
| 1. `df[var_na] = df[var].isna()` | Save a missing value dummy |
| 2. `imputation_value = df[var].mean()` | Compute a value to replace missings (or median,e tc.) |
| 3. `df['var_imp'] = df[var].fillna(imputation_value)` | Replace missings with `imputation_value` |

Notice, we didn't get rid of the original variable, and we deliberately made a missing value dummy for later analysis

## Replacing and Mapping 
- Sometimes, you just need to rewire the whole variable
- First, make a dictionary



| Method | Use |
| :---: | :---:|
| 1. `dict = {old_value:new_value,...}` | Create a dictionary of replacement values |
| 2. `df[var_map] = df[var].map(dict)` | Pass the dict to `.map()` |




## Logical Operators
- Often, we want to check a logical condition for every observation
- These are some of the most commonly used logical operators:

| Operator | Meaning | Example |
| :---: | :---:| :---:|
| `and` | and | |
|`or` | or | |
|$==$, $!=$ | equivalence, inequivalence | |
|`<=`,`<` | less-than-equal-to, less-than | |
| `in`, `not in`| set membership, set non-membership | |



## Row and Column Filtering
- We'd like to grab rows of interest and columns of interest, for many reasons.
- This is accomplished through the **locator function**, `.loc`

- Imagine we have 
    1. A logical/Boolean condition, `logical_conditional`, `like age>50` or `price<10_000`
    2. A list of variables of interest, `var_list = [var_1, var_2, ... ]`

| Method | Use |
| :---: | :---:|
| `df.loc[logical_conditional,:]` | Write the dataframe to `path` |
| `df.loc[:,var_list]` | Write the dataframe to `path` |
| `df.loc[logical_conditional,var_list]` | Both! |

## Saving the Data
- You've done all this work, how do we save it?

| Method | Use |
| :---: | :---:|
| `df.to_csv(path,index=False)` | Write the dataframe to `path.csv` |
| `df.to_parquet(path,index=False, engine='fastparquet')` | Write the dataframe to `path.parquet` |